# <font color=#003d5c>Árboles de Decisión</font>


Es un algoritmo que proporciona una arquitectura condicional de varios niveles para el análisis predictivo mediante la evaluación de la ganancia de información incremental.

<img src="images/groot.gif"></img>

Piense en el árbol de decisión como un un árbol lleno de condiciones `if-else`. La condición más eficiente se encuentra en la parte superior y disminuye gradualmente a medida que descendemos (en la mayoría de los casos).

<font color=#003d5c>**¿Por qué aprenderemos Árboles de Decisión?**</font>

- Puede aplicarse a problemas de regresión y clasificación
- Tienen muchas propiedades útiles
- Son muy populares
- Son la base de modelos más complejos
- Tienen una forma diferente de "pensar" que los otros modelos que hemos estudiado

## <font color=#003d5c>Objetivos de la lección</font>

Los estudiantes podrán:

- Explicar cómo se crea un árbol de decisión
- Construir un modelo de árbol de decisión en scikit-learn
- Tunear un modelo de árbol de decisión y  explicar cómo el tuning impacta al modelo.
- Interpretar un diagrama de árbol
- Describir las diferencias principales entre los árboles de regresión y clasificación
- Decida si un árbol de decisión es un modelo apropiado para un problema dado

Esta parte teórica se puede revisar en el octavo capítulo del libro [**Introduction to Statistical Learning**](http://www-bcf.usc.edu/~gareth/ISL/ISLR%20Seventh%20Printing.pdf).

In [ ]:
from IPython.lib.display import YouTubeVideo

In [ ]:
YouTubeVideo('qDcl-FRnwSU', width=1000,height=400)

# <font color=#003d5c>1. Árboles de Regresión</font>

Datos de un jugador de Grandes Ligas de 1986-87::

- **Años** (eje x): número de años jugando en las ligas mayores
- **Hits** (eje y): número de hits en el año anterior
- **Salario** (color): el salario bajo es azul / verde, el salario alto es rojo / amarillo

![Salary data](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/salary_color.png)

**Ejercicio grupal:**

- Los datos anteriores son nuestros **datos de entrenamiento**.
- Queremos construir un modelo que prediga el salario de **futuros jugadores** basado en años y Hits.
- Vamos a "segmentar" el grupo de características en regiones, y luego usar el **salario medio en cada región** como el salario previsto para los futuros jugadores.
- Se quiere **maximizar** la similitud (u "homogeneidad") dentro de una región determinada, y **minimizar** la similitud entre diferentes regiones.


**Reglas para segmentar:**

- Solo puedes usar **líneas rectas**, dibujadas una a la vez.
- Estas líneas deben ser verticales u horizontales.
- Una línea **se detiene** cuando golpea una línea existente.

![Salary regions](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/salary_regions.png)

Arriba están las regiones creadas por una computadora::

- $R_1$: jugadores con **menos de 5 años** de experiencia, salario medio de **\$166,000**
- $R_2$: jugadores con **5 o más años** de experiencia y **menos de 118 visitas**, salario medio de **\$403,000**
- $R_3$: jugadores con **5 o más años** de experiencia y **118 hits o más**, Salario medio de **\$846,000**

**Nota:** Años y Hits son enteros, pero la convención es usar el **punto medio** entre valores adyacentes para etiquetar una división.

Estas regiones se usan para hacer predicciones sobre **datos fuera de la muestra**. ¡Por lo tanto, solo hay tres predicciones posibles! (¿Es esto diferente de cómo **la regresión lineal** hace predicciones?)

A continuación se muestra el árbol de regresión equivalente:

![Salary tree](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/salary_tree.png)

La primera división es **Años <4.5**, por lo que esa división va en la parte superior del árbol. Cuando una regla de división es **Verdadero**, sigue la rama izquierda. Cuando una regla de división es **Falsa**, sigue la rama derecha.

Para los jugadores en la **rama izquierda**, el Salario medio es \$166,000, por lo que se etiqueta con ese valor. (El salario se ha dividido por 1000 y se ha transformado a 5.11 usando el logaritmo).

Para jugadores en la **rama derecha**, hay una división adicional en **Hits <117.5**, dividiendo jugadores en dos regiones Salario más: \$403,000 (transformado a 6.00), y \$846,000 (transformado a 6.74).

![Salary tree annotated](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/salary_tree_annotated.png)

**¿Qué le dice este árbol acerca de sus datos?¿Qué insights puedes obtener?**

- Años es el factor más importante que determina el Salario, con un menor número de años correspondientes a un Salario inferior.
- Para un jugador con un número menor de años, Hits no es un factor importante que determina el salario.
- Para un jugador con un mayor número de años, Hits es un factor importante que determina el salario, con un mayor número de Hits correspondientes a un salario más alto.

**Pregunta del Millón:** ¿Qué le gusta y qué le desagrada de los árboles de decisión hasta el momento?

## <font color=#003d5c>I. Construir un árbol de regresión a mano</font>

Los **datos de entrenamiento** están en un pequeño dataset de [precios de venta de vehículos usados](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/vehicles_train.csv). Su meta es **predecir el precio** para los datos de prueba.

1. Lea los datos en un DataFrame.
2. Explore los datos por medio de gráficas, o agrupaciones (recordar el uso del `group_by`).
3. Decida qué característica es el predictor más importante y utilícela para crear su primera regla de división.
    - Solo se permiten divisiones binarias.
4. Después de hacer su primera división, divida su DataFrame en dos partes y luego explore cada parte para descubrir qué otras divisiones hacer.
5. Deje de hacer divisiones una vez que esté convencido de que logra un buen equilibrio entre el ajuste insuficiente (underfitting) y el ajuste excesivo (overfitting).
    - Tu objetivo es construir un modelo que generalice bien.
    - ¡Puedes dividir la misma variable varias veces!
6. Dibuje su árbol, etiquetando las hojas con el precio promedio de las observaciones en esa región.
    - Asegúrate de que nada esté al revés: sigue la **rama izquierda** si la regla es verdadera, y la **rama derecha** si la regla es falsa.

## <font color=#003d5c>¿Cómo construye una computadora un árbol de regresión?</font>

**Enfoque ideal:** Considera cada posible partición del grupo de características (no factible computacionalmente)

**Enfoque "suficientemente bueno":** división binaria recursiva

1. Comienza en la parte superior del árbol
2. Para **cada característica**, examina **todos los puntos de corte posibles**, y elige la característica y el punto de corte para que el árbol resultante tenga el error cuadrático medio (MSE) más bajo posible.
3. Examina las dos regiones resultantes, y nuevamente haga una **división simple** (en una de las regiones) para minimizar el MSE.
4. Continúa repitiendo el paso 3 hasta que se cumpla un **criterio de detención**:
    - profundidad máxima del árbol (número máximo de divisiones requeridas para llegar a una hoja)
    - número mínimo de observaciones en una hoja

### <font color=#003d5c>Aplicación de la lógica: Elegir el punto de corte ideal para una característica dada</font>

In [ ]:
import pandas as pd

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
url = 'data/vehicles_train.csv'
train = pd.read_csv(url, encoding="latin1")

In [ ]:
train.head()

In [ ]:
train["vtype"].value_counts()

In [ ]:
# Codificar "car" como 0 y "truck" como 1
train['vtype'] = train.vtype.map({'car':0, 'truck':1})

In [ ]:
train

In [ ]:
# Antes de hacer las divisiones, calculemos la media de todo el dataset para que sea nuestra valor de predicción
train['prediction_sin_reglas'] = train.price.mean()
train

**<font color=#003d5c>Calcular el error (MSE):</font>**

In [ ]:
(((train['price'] - train['prediction_sin_reglas'])**2).mean()) ** 0.5

In [ ]:
train.columns

**<font color=#003d5c>a) Elegir nuestra primera mejor división:</font>**
<br><br>
Por ejemplo dividamos todo el dataset por la variable `year` y el valor de `2010` y calculemos los promedios del precio en las 2 regiones obtenidas y asignarles como valor de predicción en la variable `pred`.

In [ ]:
year = 2010

In [ ]:
train.loc[train.year < year, :]

In [ ]:
train.loc[train.year >= year, :]

In [ ]:
train.loc[train.year < year, 'pred_año_2010'] = train.loc[train.year < year, 'price'].mean()
train.loc[train.year >=year, 'pred_año_2010'] = train.loc[train.year >=year, 'price'].mean()

In [ ]:
train

In [ ]:
(((train['price'] - train['pred_año_2010'])**2).mean()) ** 0.5

<font color=#003d5c>Región de la Izquierda:</font>

In [ ]:
train_izq = train.loc[train.year<2010].copy()

In [ ]:
train_izq.year.unique()

<font color=#003d5c>Región de la Derecha:</font>

In [ ]:
train_der = train.loc[train.year>=2010].copy()

In [ ]:
train_der.year.unique()

Definamos una función que nos permite calcular el error obtenido por la división en 2 regiones en base a un valor de la variable `year`.

In [ ]:
def error_año(train, year):
    train.loc[train.year <year, 'pred'] = train.loc[train.year <year, 'price'].mean()
    train.loc[train.year>=year, 'pred'] = train.loc[train.year>=year, 'price'].mean()
    return((((train['price'] - train['pred'])**2).mean()) ** 0.5)

In [ ]:
error_min = float('inf')
year_split = 0
for year in train.year.unique():
    error_año_var = error_año(train, year)
    print(year, ": ", error_año_var)
    if error_año_var < error_min:
        error_min = error_año_var
        year_split = year
print("Se debe hacer la división por el valor de ", year_split, "ya que genera 2 regiones que nos permiten obtener un error de ", error_min)

**<font color=#003d5c>b) Evaluar el punto de corte con el MSE más bajo:</font>**

¿Pero es correcto que hayamos elegido la variable `year`?

In [ ]:
def error_split(train, var, value):
    train.loc[train[var] <value, 'pred'] = train.loc[train[var] <value, 'price'].mean()
    train.loc[train[var]>=value, 'pred'] = train.loc[train[var]>=value, 'price'].mean()
    return ((((train['price'] - train['pred'])**2).mean()) ** 0.5)

In [ ]:
error_min, var_final, val_final = float('inf'), '', 0
for var_ in ['year', 'miles', 'doors', 'vtype']:
    print('-------------{}-------------'.format(var_))
    for val_ in train[var_].unique():
        error = error_split(train, var_, val_)
        print(val_, ": ", error)
        if error < error_min:
            error_min = error
            var_final, val_final = var_, val_
print("\n--------------------------------------------------")
print("Se debe hacer la división por el valor de ", val_final, " de la variable ", var_final, " ya que genera 2 regiones que nos permiten obtener un error de ", error_min)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (10, 4)
plt.rcParams['font.size'] = 14

In [ ]:
# Graficar la variable 'year' (eje-X) versus RMSE (eje-y)
values = train['year'].unique()
errors = [error_split(train, 'year', v) for v in values]
plt.plot(values, errors)
plt.xlabel('Punto de corte en "year"')
plt.ylabel('RMSE (más pequeño es mejor)')

**<font color=#003d5c>c) Examina las 2 regiones resultantes:</font>**

In [ ]:
train_izq = train.loc[train['year'] <2007, :]
train_der = train.loc[train['year']>=2007, :]

In [ ]:
train_izq.head()

**<font color=#003d5c>d) Evaluar si se cumple criterio de detención:</font>**

In [ ]:
train_izq.shape

In [ ]:
train_der.shape

**<font color=#003d5c>Continuar el Ciclo:</font>**
<br><br>Con la región de la Izquierda

In [ ]:
error_min, var_final, val_final = float('inf'), '', 0
for var_ in ['year', 'miles', 'doors', 'vtype']:
    print('-------------{}-------------'.format(var_))
    for val_ in train_izq[var_].unique():
        error = error_split(train_izq, var_, val_)
        print(val_, ": ", error)
        if error < error_min:
            error_min = error
            var_final, val_final = var_, val_
print("\n--------------------------------------------------")
print("Se debe hacer la división por el valor de ", val_final, " de la variable ", var_final, " ya que genera 2 regiones que nos permiten obtener un error de ", error_min)

In [ ]:
train_izq_izq = train_izq.loc[train_izq['miles']< 138000]
train_izq_der = train_izq.loc[train_izq['miles']>=138000]

In [ ]:
print(train_izq_izq.shape)
print(train_izq_der.shape)

Con la región de la Derecha

In [ ]:
error_min, var_final, val_final = float('inf'), '', 0
for var_ in ['year', 'miles', 'doors', 'vtype']:
    print('-------------{}-------------'.format(var_))
    for val_ in train_der[var_].unique():
        error = error_split(train_der, var_, val_)
        print(val_, ": ", error)
        if error < error_min:
            error_min = error
            var_final, val_final = var_, val_
print("\n--------------------------------------------------")
print("Se debe hacer la división por el valor de ", val_final, " de la variable ", var_final, " ya que genera 2 regiones que nos permiten obtener un error de ", error_min)

In [ ]:
train_der_izq = train_der.loc[train_der['year']< 2012]
train_der_der = train_der.loc[train_der['year']>=2012]

In [ ]:
print(train_der_izq.shape)
print(train_der_der.shape)

In [ ]:
train_der_izq = train_der.loc[train_der['miles']< 30000]
train_der_der = train_der.loc[train_der['miles']>=30000]

In [ ]:
print(train_der_izq.shape)
print(train_der_der.shape)

<div class="alert alert-info" role="alert">
  <strong>Recordar:</strong> Antes de cada división, este proceso se repite para cada característica, y se eligen la característica y el punto de corte que produce la `RMSE` más baja.
</div>

## <font color=#003d5c>II. Construyendo un árbol de regresión en scikit-learn</font>

Definir nuestro dataset de Entrenamiento y el Target

In [ ]:
predictores = ['year', 'miles', 'doors', 'vtype']
X = train[predictores]
y = train.price

In [ ]:
# Instanciar un objeto de la clase DecisionTreeRegressor (con random_state=1)
from sklearn.tree import DecisionTreeRegressor
treereg = DecisionTreeRegressor(random_state=1, max_depth=2, criterion='mse')
treereg

In [ ]:
treereg.fit(X, y)

Visualizar el Árbol de Regresión

In [ ]:
from graphviz import Source
from sklearn.tree import export_graphviz
from IPython.display import SVG

In [ ]:
#graph = Source( export_graphviz(treereg, out_file=None, feature_names=predictores, filled=True))
#SVG(graph.pipe(format='svg'))

Leyendo los nodos internos:

- **samples:** cantidad de observaciones en ese nodo antes de dividir
- **mse:** MSE calculado al comparar los valores de respuesta reales en ese nodo con el valor de respuesta promedio en ese nodo
- **regla:** regla utilizada para dividir ese nodo (ir a la izquierda si es verdadero, ir a la derecha si es falso)

Leyendo las hojas:

- **samples:** cantidad de observaciones en ese nodo
- **value:** valor de respuesta promedio en ese nodo
- **mse:** MSE calculado al comparar los valores de respuesta reales en ese nodo contra "value"

## <font color=#003d5c>III. Tunning en un árbol de regresión usando scikit-learn</font>

```python
Usando  for
```

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error

In [ ]:
error_list = []
for max_depth in range(1, 9):
    treereg = DecisionTreeRegressor(random_state=1, max_depth=max_depth, criterion='mse')
    treereg.fit(X, y)
    error = np.sqrt(mean_squared_error(y, treereg.predict(X)))
    error_list.append(error)
    print('max_depth={}, error={}'.format(max_depth, error))

In [ ]:
# Graficar la variable 'year' (eje-X) versus RMSE (eje-y)
values = range(1, 9)
plt.plot(values, error_list)
plt.xlabel('max_depth')
plt.ylabel('RMSE (más pequeño es mejor)')

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
treereg = DecisionTreeRegressor(random_state=1, max_depth=max_depth, criterion='mse')
parameters = {'max_depth': list(range(1, 9))}

In [ ]:
clf = GridSearchCV(treereg, parameters)

In [ ]:
clf.fit(X, y)

In [ ]:
clf.best_estimator_

In [ ]:
import numpy as np
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(treereg, X, y, cv=14, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

## <font color=#003d5c>¿Qué sucede cuando generamos un árbol demasiado profundo?</font>

- Izquierda: Árbol de regresión para Salario **crecimiento más profundo**
- Derecha: Comparación de **errores de entrenamiento, prueba y validación cruzada** para árboles con diferentes números de hojas

![Salary tree grown deep](https://github.com/justmarkham/DAT8/raw/226791169b1cc6df8e8845c12e34e748d5ffaa85/notebooks/images/salary_tree_deep.png)

El **error de entrenamiento** continúa disminuyendo a medida que aumenta el tamaño del árbol (debido al sobreajuste), pero el error **de validación cruzada** más bajo ocurre para un árbol con 3 hojas.

## Tuning: Ajustando un árbol de regresión

Tratemos de reducir el RMSE ajustando el parámetro **max_depth**:

In [ ]:
# prueba diferentes valores uno a uno
treereg = DecisionTreeRegressor(max_depth=1, random_state=1)
scores = cross_val_score(treereg, X, y, cv=14, scoring='neg_mean_squared_error')
np.mean(np.sqrt(-scores))

O bien, podríamos hacer un bucle para probar un rango de valores:

In [ ]:
# lista de valores para probar
max_depth_range = range(1, 8)

# lista para almacenar el RMSE promedio para cada valor de max_depth
RMSE_scores = []

# use LOOCV con cada valor de max_depth
for depth in max_depth_range:
    treereg = DecisionTreeRegressor(max_depth=depth, random_state=1)
    MSE_scores = cross_val_score(treereg, X, y, cv=14, scoring='neg_mean_squared_error')
    RMSE_scores.append(np.mean(np.sqrt(-MSE_scores)))

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
# plot max_depth (x-axis) versus RMSE (y-axis)
plt.plot(max_depth_range, RMSE_scores)
plt.xlabel('max_depth')
plt.ylabel('RMSE (lower is better)')

In [ ]:
# max_depth = 3 fue el mejor, así que ajuste un árbol usando ese parámetro
treereg = DecisionTreeRegressor(max_depth=3, random_state=1)
treereg.fit(X, y)

In [ ]:
# "Importancia de Gini" de cada característica: la reducción total de errores (normalizada) provocada por esa característica
pd.DataFrame({'feature':predictores, 'importance':treereg.feature_importances_})

## Hacer predicciones para los datos de prueba

In [ ]:
# Leer la data de prueba
url = 'data/vehicles_test.csv'
test = pd.read_csv(url, encoding="latin1")
test['vtype'] = test.vtype.map({'car':0, 'truck':1})
test




**Pregunta: ** Usando el diagrama de árbol de arriba, ¿qué predicciones hará el modelo para cada observación?

In [ ]:
# utilizar el modelo ajustado para hacer predicciones sobre los datos de prueba
X_test = test[predictores]
y_test = test.price
y_pred = treereg.predict(X_test)
y_pred

In [ ]:
# calcular RMSE
np.sqrt(mean_squared_error(y_test, y_pred))

# <font color=#003d5c>2. Árboles de clasificación</font>
Una pequeña Introducción en la siguiente página: <a href="http://www.r2d3.us/una-introduccion-visual-al-machine-learning-1/">http://www.r2d3.us/</a>

## Comparación de árboles de regresión y árboles de clasificación

|Árbol de regresión|Árbol de clasificación|
|---|---|
|Predice una respuesta continua|Predice una respuesta categórica|
|Predice usando la respuesta media de cada hoja|Predice usando la clase más común en cada hoja|
|Las divisiones se eligen para minimizar MSE|Las divisiones se eligen para minimizar el índice de Gini (discutido a continuación)|

## Criterios de división para los árboles de clasificación

Opciones comunes para los criterios de división:

- **Tasa de error de clasificación:** fracción de observaciones de entrenamiento en una región que no pertenece a la clase más común
- **Índice de Gini:** medida de la varianza total entre clases en una región
- **Entropía:** medida del desorden. La entropía es un indicador de lo desordenado que son tus datos.
<img src="https://sebastianraschka.com/images/faq/decision-tree-binary/overview-plot.png">

### Ejemplo de tasa de error de clasificación

Imagina que estamos prediciendo si alguien compra un iPhone o un Android:

- En un nodo en particular, hay **25 observaciones** (compradores de teléfonos), de los cuales **10 compraron iPhones y 15 compraron Android**.
- Dado que la clase mayoritaria es **Android**, esa es nuestra predicción para las 25 observaciones, y por lo tanto la tasa de error de clasificación es **10/25 = 40%**.

Nuestro objetivo al hacer splits es **reducir la tasa de error de clasificación**. Probemos dividir en género:

- **Hombres:** 2 iPhones y 12 Androids, por lo tanto, la clase predicha es Android. (2 equivocaciones)
- **Mujeres:** 8 iPhones y 3 Androids, por lo que la clase prevista es iPhone. (3 equivocaciones)
- Tasa de error de clasificación después de esta división sería **5/25 = 20%**

Compare eso con una división en la edad:

- **30 o menos:** 4 iPhones y 8 Androids, por lo tanto, la clase predicha es Android. (4 equivocaciones)
- **31 o más:** 6 iPhones y 7 Androids, por lo tanto, la clase predicha es Android. (6 equivocaciones)
- Tasa de error de clasificación después de esta división sería **10/25 = 40%**

El algoritmo del árbol de decisión intentará **cada división posible en todas las características**, y elegirá la división que más **reduzca la tasa de error.**

### Ejemplo de Índice de Gini

Calcule el índice de Gini antes de hacer una división:

$$1 - \left(\frac {iPhone} {Total}\right)^2 - \left(\frac {Android} {Total}\right)^2 = 1 - \left(\frac {10} {25}\right)^2 - \left(\frac {15} {25}\right)^2 = 0.48$$

- El **valor máximo** del índice de Gini es 0.5, y ocurre cuando las clases están perfectamente balanceadas en un nodo.
- El **valor mínimo** del índice de Gini es 0, y ocurre cuando solo hay una clase representada en un nodo.
- Se dice que un nodo con **un índice de Gini más bajo es más "puro"**.

Evaluar la división en ** sexo ** usando el índice de Gini:

$$\text{Hombres: } 1 - \left(\frac {2} {14}\right)^2 - \left(\frac {12} {14}\right)^2 = 0.24$$
$$\text{Mujeres: } 1 - \left(\frac {8} {11}\right)^2 - \left(\frac {3} {11}\right)^2 = 0.40$$
$$\text{Peso Promedio: } 0.24 \left(\frac {14} {25}\right) + 0.40 \left(\frac {11} {25}\right) = 0.31$$

Evaluar la división en **edad** usando el índice de Gini:

$$\text{30 o joven: } 1 - \left(\frac {4} {12}\right)^2 - \left(\frac {8} {12}\right)^2 = 0.44$$
$$\text{31 o viejo: } 1 - \left(\frac {6} {13}\right)^2 - \left(\frac {7} {13}\right)^2 = 0.50$$
$$\text{Peso Promedio: } 0.44 \left(\frac {12} {25}\right) + 0.50 \left(\frac {13} {25}\right) = 0.47$$

De nuevo, el algoritmo del árbol de decisiones intentará **cada posible división**, y elegirá la división que **reduce el índice de Gini (y así aumenta la "pureza del nodo") más.**

### Comparando la tasa de error de clasificación y el índice de Gini

- Generalmente se prefiere el índice de Gini porque hará divisiones que **aumentan la pureza del nodo**, incluso si esa división no cambia la tasa de error de clasificación.
- La pureza del nodo es importante porque nos interesan las **proporciones de clase** en cada región, ya que así es como calculamos la **probabilidad pronosticada** de cada clase.
- Los criterios de división predeterminados de scikit-learn para los árboles de clasificación son el índice de Gini.

Nota: Hay otro criterio de división común llamado **entropía cruzada**. Es numéricamente similar al índice de Gini, pero más lento de calcular, por lo tanto, no es tan popular como el índice de Gini. Queda como tarea revisarla.

## Construyendo un árbol de clasificación en scikit-learn

Construiremos un árbol de clasificación usando el dataset de Crédito Hipotecario:

In [ ]:
# Lectura del dataset
df = pd.read_csv('data/DS_Credito Hipotecario.csv')

# Preprocesamiento de la data
df['SEXO'].fillna(df['SEXO'].mode()[0], inplace=True)
df['FLAG_CASADO'].fillna(df['FLAG_CASADO'].mode()[0], inplace=True)
df['NRO_DEPENDIENTES'].fillna(df['NRO_DEPENDIENTES'].mode()[0], inplace=True)
df['FLAG_TRAB_INDEP'].fillna(df['FLAG_TRAB_INDEP'].mode()[0], inplace=True)
df['INGRESOS_COSOLICITANTE'].fillna(df['INGRESOS_COSOLICITANTE'].mean(), inplace=True)
df['MONTO_PRESTAMO_MILES'].fillna(df['MONTO_PRESTAMO_MILES'].mean(), inplace=True)
df['PLAZO_PRESTAMO_MESES'].fillna(df['PLAZO_PRESTAMO_MESES'].mean(), inplace=True)
df['FLAG_HISTORIAL_CREDITICIO'].fillna(df['FLAG_HISTORIAL_CREDITICIO'].mode()[0], inplace=True)

df['SEXO'] = df['SEXO'].map({'Male': 0, 'Female': 1})
df['FLAG_CASADO'] = df['FLAG_CASADO'].map({'No': 0, 'Yes': 1})
df['EDUCACION'] = df['EDUCACION'].map({'Not Graduate': 0, 'Graduate': 1})
df['FLAG_TRAB_INDEP'] = df['FLAG_TRAB_INDEP'].map({'No': 0, 'Yes': 1})
df['FLAG_HISTORIAL_CREDITICIO'] = df['FLAG_HISTORIAL_CREDITICIO'].map({'Malo': 0, 'Bueno': 1})
df = pd.concat([df, pd.get_dummies(df['TIPO_ZONA'], prefix = 'TIPO_ZONA', drop_first = True)], axis=1)
df = pd.concat([df, pd.get_dummies(df['NRO_DEPENDIENTES'], prefix = 'NRO_DEPENDIENTES', drop_first = True)], axis=1)
del df['TIPO_ZONA']
del df['NRO_DEPENDIENTES']
df['FLAG_CRED_HIPO'] = df['FLAG_CRED_HIPO'].map({'Y': 1, 'N': 0})

In [ ]:
df.head()

- **FLAG_CRED_HIPO:** 0=No al CH, 1=Sí al CH (Target)
- **EDUCACION:** 0=No Graduado, 1=Graduado
    - ¿Qué pasará si el árbol se divide en esta característica?
- **SEXO:** 0=hombre, 1=mujer
- **INGRESOS_SOLICITANTE:** valor numérico
- **FLAG_HISTORIAL_CREDITICIO:** Bueno o Malo

In [ ]:
# Definir X e Y
predictores = ['SEXO', 'FLAG_CASADO', 'EDUCACION', 'FLAG_TRAB_INDEP', 'INGRESOS_SOLICITANTE', 'INGRESOS_COSOLICITANTE',
                'MONTO_PRESTAMO_MILES', 'PLAZO_PRESTAMO_MESES','FLAG_HISTORIAL_CREDITICIO', 'TIPO_ZONA_Semiurban',
                'TIPO_ZONA_Urban', 'NRO_DEPENDIENTES_1', 'NRO_DEPENDIENTES_2','NRO_DEPENDIENTES_3+']
X = df[predictores]
y = df.FLAG_CRED_HIPO

In [ ]:
# Entrenar un árbol de decisión con max_depth=3 con todos los datos
from sklearn.tree import DecisionTreeClassifier
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X, y)

Para poder visualizar la estructura del árbol necesitan tener instalado lo siguiente:

Librería de Python Graphviz. Ejecutar **pip install graphviz** en la ventana de línea de comandos. <br>
Software Graphviz (Graph Visualization). Lo pueden descargar desde este enlace: [LINK](https://graphviz.gitlab.io/_pages/Download/Download_windows.html)<br>
**Nota**: Recordar que deben referenciarla en variables de entorno. Es decir, copiar ruta "C:\Program Files (x86)\Graphviz2.38\bin" (verificar que esta ruta exista en su portátil después de la instalación) en la variable de entorno del sistema "Path"

In [ ]:
#graph = Source(export_graphviz(treeclf, out_file=None, feature_names=predictores, filled=True))
#SVG(graph.pipe(format='svg'))

In [ ]:
from IPython.display import SVG
from IPython.display import display                               
from ipywidgets import interactive

def plot_tree(crit, split, depth, min_split, min_leaf=0.2):
    estimator = DecisionTreeClassifier(random_state = 0 
          , criterion = crit
          , splitter = split
          , max_depth = depth
          , min_samples_split=min_split
          , min_samples_leaf=min_leaf)
    estimator.fit(X, y)
    graph = Source(export_graphviz(estimator
          , out_file=None
          , feature_names=predictores
          , class_names=['0', '1', '2']
          , filled = True))

    display(SVG(graph.pipe(format='svg')))

    return estimator

In [ ]:
"""inter=interactive(plot_tree 
   , crit = ["gini", "entropy"]
   , split = ["best", "random"]
   , depth=[1,2,3,4]
   , min_split=(0.1,1)
   , min_leaf=(0.1,0.5))
display(inter)"""

In [ ]:
df.shape

Distribución del Target en el primer nivel

In [ ]:
df.groupby('FLAG_CRED_HIPO').count()

Cálculo de Gini

In [ ]:
gini = 1 - (192.0/614)**2 - (422/614)**2
round(gini, 3)

Región de la Izquierda

In [ ]:
df_izq = df.loc[df['FLAG_HISTORIAL_CREDITICIO'] <= 0.5]

In [ ]:
df_izq.groupby('FLAG_CRED_HIPO').count()

In [ ]:
gini_izq = 1 - (82.0/89)**2 - (7.0/89)**2
round(gini_izq, 3)

Región de la Derecha

In [ ]:
df_der = df.loc[df['FLAG_HISTORIAL_CREDITICIO'] > 0.5]

In [ ]:
df_der.groupby('FLAG_CRED_HIPO').count()

In [ ]:
gini_der = 1 - (110.0/525)**2 - (415.0/525)**2
round(gini_der, 3)

In [ ]:
gini_2nd_nivel = gini_izq * (89/ 614) + gini_der * (525/ 614)
gini_2nd_nivel

### ¿Por qué Gini y no Error de Clasificación?

<img src="images/decisionTreeMissClassification.png">

Observe la división en esta parte del árbol: la **misma clase** se predice en sus dos hojas. Esa división no afectó la **tasa de error de clasificación**, aunque aumentó la **pureza del nodo**, que es importante porque aumenta la precisión de nuestras probabilidades predichas.

In [ ]:
# calculando la importancia de variables
pd.DataFrame({'feature':predictores, 'importance':treeclf.feature_importances_}).sort_values(by='importance', ascending=False)

### Split Train - Test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
train_accuracy_list, train_auc_list = [], []
test_accuracy_list, test_auc_list = [], []
max_depth_range = list(range(1, 30))
for max_depth in max_depth_range:
    
    treeclf = DecisionTreeClassifier(max_depth=max_depth, random_state=1)
    treeclf.fit(X_train, y_train)
    
    y_train_pred = treeclf.predict(X_train)
    y_test_pred = treeclf.predict(X_test)
    
    y_train_pred_proba = treeclf.predict_proba(X_train)[:, 1]
    y_test_pred_proba = treeclf.predict_proba(X_test)[:, 1]
    
    accuracy_train = accuracy_score(y_train, y_train_pred)
    auc_train = roc_auc_score(y_train, y_train_pred_proba)
    
    accuracy_test = accuracy_score(y_test, y_test_pred)
    auc_test = roc_auc_score(y_test, y_test_pred_proba)
    
    train_accuracy_list.append(accuracy_train)
    train_auc_list.append(auc_train)
    test_accuracy_list.append(accuracy_test)
    test_auc_list.append(auc_test)
    
    print("Train - max_depth: {}, accuracy: {}, auc: {}".format(max_depth, accuracy_train, auc_train))
    print("Test - max_depth: {}, accuracy: {}, auc: {} \n".format(max_depth, accuracy_test, auc_test))

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(15, 5))
ax[0].plot(max_depth_range, train_accuracy_list, c='orange')
ax[0].plot(max_depth_range, test_accuracy_list,  c='skyblue')
ax[1].plot(max_depth_range, train_auc_list, c='orange')
ax[1].plot(max_depth_range, test_auc_list,  c='skyblue')
ax[0].set_title('Accuracy evaluation')
ax[0].set_xlabel('max_depth')
ax[0].set_ylabel('Accuracy')
ax[1].set_title('AUC Score evaluation')
ax[1].set_xlabel('max_depth')
ax[1].set_ylabel('AUC Score')

## <font color=#003d5c>Aplicar al Dataset de Attrition</font>

In [ ]:
import pandas as pd

In [ ]:
# Cargamos nuestra data de Attrition
df_train_clientes = pd.read_excel('data/train_clientes.xlsx')

In [ ]:
df_train_clientes.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
# Seleccionamos las variables de  tipo "objeto"
columns_cat = df_train_clientes.select_dtypes(include=['object']).columns
labelEncoders = {}

# Hacemos el Label Encoding
for col in columns_cat:
    labelEncoders[col] = LabelEncoder()
    df_train_clientes['{0}_ENC'.format(col)] = labelEncoders[col].fit_transform(df_train_clientes[col].fillna('na'))

In [ ]:
# Eliminamos las variables del tipo object
df_train_clientes.drop(axis=1, labels=columns_cat, inplace=True)

In [ ]:
# Función para reducir la memoria que ocupa un DataFrame
def optimizar(df):
    float_cols = df.select_dtypes(include=['float'])
    int_cols = df.select_dtypes(include=['integer'])

    for var_f in float_cols.columns:
        df[var_f] = pd.to_numeric(df[var_f], downcast='float')
    for var_i in int_cols.columns:
        df[var_i] = pd.to_numeric(df[var_i], downcast='integer')
    
    return df

In [ ]:
# Optimizamos la memoria del DataFrame
df_train_clientes = optimizar(df_train_clientes)

In [ ]:
# Definir los predictores (variables independientes)
predictores = [c for c in df_train_clientes.columns if c not in ["ID_CORRELATIVO", "CODMES", "ATTRITION"]]

In [ ]:
# Definir los DataFrames con las variables y el del Target
X = df_train_clientes[predictores]
y = df_train_clientes["ATTRITION"]

In [ ]:
# importar el métofo train_test_split del sub-módulo model_selection de Scikit-learn
from sklearn.model_selection import train_test_split

In [ ]:
# Selección de muestra de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train_fillna = X_train.copy()
X_train_fillna["EDAD"].fillna(value=X_train_fillna["EDAD"].mean(), inplace=True)
X_train_fillna["ANTIGUEDAD"].fillna(value=X_train_fillna["ANTIGUEDAD"].mean(), inplace=True)
X_val_fillna = X_val.copy()
X_val_fillna["EDAD"].fillna(value=X_train_fillna["EDAD"].mean(), inplace=True)
X_val_fillna["ANTIGUEDAD"].fillna(value=X_train_fillna["ANTIGUEDAD"].mean(), inplace=True)

In [ ]:
treeclf = DecisionTreeClassifier(max_depth=3, random_state=1)
treeclf.fit(X_train_fillna, y_train)

In [ ]:
#graph = Source( export_graphviz(treeclf, out_file=None, feature_names=predictores, filled=True))
#SVG(graph.pipe(format='svg'))

In [ ]:
from IPython.display import SVG
from IPython.display import display                               
from ipywidgets import interactive

def plot_tree(crit, split, depth, min_split, min_leaf=0.2):
    estimator = DecisionTreeClassifier(random_state = 0 
          , criterion = crit
          , splitter = split
          , max_depth = depth
          , min_samples_split=min_split
          , min_samples_leaf=min_leaf)
    estimator.fit(X_train_fillna, y_train)
    graph = Source(export_graphviz(estimator
          , out_file=None
          , feature_names=predictores
          , class_names=['0', '1', '2']
          , filled = True))

    display(SVG(graph.pipe(format='svg')))

    return estimator

In [ ]:
"""inter=interactive(plot_tree 
   , crit = ["gini", "entropy"]
   , split = ["best", "random"]
   , depth=[1,2,3,4]
   , min_split=(0.1,1)
   , min_leaf=(0.1,0.5))
display(inter)"""

# <font color=#003d5c>3. Comparando árboles de decisión con otros modelos</font>

**Ventajas de los árboles de decisión:**

- Puede usarse para regresión o clasificación
- Se puede mostrar gráficamente
- Altamente interpretable
- Se puede especificar como una serie de reglas y una toma de decisiones humana más cercana a la de otros modelos
- La predicción es rápida
- Las variables no necesitan escala
- Aprende automáticamente las interacciones de las variables
- Tiende a ignorar variables irrelevantes
- No paramétrico (superará a los modelos lineales si la relación entre las características y la respuesta es altamente no lineal)

<img src="images/groot_advantages.gif"></img>

**Desventajas de los árboles de decisión:**

- El rendimiento es (generalmente) no competitivo con los mejores métodos de aprendizaje supervisado
- Puede sobredimensionar fácilmente los datos de entrenamiento (se requiere ajuste)
- Pequeñas variaciones en los datos pueden resultar en un árbol completamente diferente (alta varianza)
- La división binaria recursiva toma decisiones "localmente óptimas" que pueden no dar como resultado un árbol globalmente óptimo
- No tiende a funcionar bien si las clases son altamente desequilibradas
- No tiende a funcionar bien con conjuntos de datos muy pequeños

<img src="images/groot_disadvantages.gif"></img>

**Referencias**:
- http://nbviewer.jupyter.org/github/albahnsen/PracticalMachineLearningClass/blob/master/notebooks/13_decision_trees.ipynb
- https://lethalbrains.com/learn-ml-algorithms-by-coding-decision-trees-439ac503c9a4